In [ ]:
'''
Analyzing GIT repositories based on commit history

Questions answered:
1. How many developers test a single app integration? 
2. How many apps does a single developer test?
'''

In [44]:
from pydriller import Repository
import json
from tqdm import tqdm
import pandas as pd
import regex as re

In [11]:
# Specify the path to your Git repository
repo_path = 'https://github.com/home-assistant/core'

# Create a dictionary to store developer names and the files they affected
developers_commited_files = {}

# Create a Repository object
repo = Repository(repo_path)

# Iterate over all commits in the repository
for commit in repo.traverse_commits():
    author_name = commit.author.name
    affected_files = commit.modified_files

    # Update the files affected by each developer
    if author_name in developers_commited_files:
        developers_commited_files[author_name].extend([modification.new_path for modification in affected_files])
    else:
        developers_commited_files[author_name] = [modification.new_path for modification in affected_files]

# Print the developers and the files they affected
for developer, files in developers_commited_files.items():
    print(f"{developer} affected {len(files)} files:")
    for file_path in files:
        print(f" - {file_path}")


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)




 - homeassistant/components/fibaro/translations/el.json
 - homeassistant/components/fibaro/translations/en.json
 - homeassistant/components/fibaro/translations/et.json
 - homeassistant/components/fibaro/translations/fr.json
 - homeassistant/components/fibaro/translations/he.json
 - homeassistant/components/fibaro/translations/hu.json
 - homeassistant/components/fibaro/translations/id.json
 - homeassistant/components/fibaro/translations/it.json
 - homeassistant/components/fibaro/translations/ja.json
 - homeassistant/components/fibaro/translations/nl.json
 - homeassistant/components/fibaro/translations/no.json
 - homeassistant/components/fibaro/translations/pl.json
 - homeassistant/components/fibaro/translations/pt-BR.json
 - homeassistant/components/fibaro/translations/ru.json
 - homeassistant/components/fibaro/translations/tr.json
 - homeassistant/components/fibaro/translations/zh-Hant.json
 - homeassistant/components/filesize/translations/bg.json
 - homeassistant/components/filesize/

In [18]:
json_file_path = "dev_commit_files.json"

# Write the dictionary to a JSON file
with open(json_file_path, "w") as json_file:
    json.dump(developers_files, json_file, indent=4)

In [32]:
###################################
#GET JSON FROM FILE
###################################
developers_commited_files = json.load(open('dev_commit_files.json'))

 

In [33]:
dev_tests_app={}
for dev_name in tqdm(developers_commited_files.keys()):
    dev_tests_app[dev_name]=[]
    for file in developers_commited_files[dev_name]:
        if file and file.startswith("tests/components"):
            #print(file)
            if len(file.split("/"))>3:
                app=file.split("/")[2]
                #print(file.split("/")[2])
            else:
                app="Common_Tests"
                #print("Common test file: "+file)
            if app not in dev_tests_app[dev_name]:
                dev_tests_app[dev_name].append(app)

100%|████████████████████████████████████| 3563/3563 [00:00<00:00, 74771.62it/s]


In [43]:
# Write the dictionary to a JSON file
with open("dev_tests_apps.json", "w") as json_file:
    json.dump(dev_tests_app, json_file, indent=4)

In [45]:
dev_apps=pd.DataFrame([],columns=["dev","# of apps tested"])
for dev in dev_tests_app.keys():
    dev_name=re.sub(r'\W+', '', dev)
    dev_apps.loc[len(dev_apps)]=[dev_name,len(dev_tests_app[dev])]   
dev_apps.to_csv("dev_tests_apps.csv")

In [48]:
app_testedBy_devs={}
for dev, apps in dev_tests_app.items():
    for app in apps:
        if app not in app_testedBy_devs:
            app_testedBy_devs[app] = [dev]
        else:
            app_testedBy_devs[app].append(dev)
len(app_testedBy_devs)

854

In [49]:
app_devs=pd.DataFrame([],columns=["app","# of devs committed"])
for app in app_testedBy_devs.keys():
    app_devs.loc[len(app_devs)]=[app,len(app_testedBy_devs[app])]   
app_devs.to_csv("app_testedBy_devs.csv")